# Testing the DEB model
## Santruckova et al. (2004)
1. Importing libraries

In [46]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from scipy.optimize import dual_annealing
from scipy.optimize import differential_evolution

2. Defining DEB model

In [47]:
def DEBmodel (y, t, pars):
    #define initial pools
    S=y[0];    e=y[1];    X1=y[2];     CO2=y[3]
    #define parameters
    yA=pars[0]; 
    Km=pars[1];     
    v=pars[2];
    m=pars[3]; 
    g=pars[4]; 
    ce=pars[5];
    MX1=ce/4;
    #Define fluxes
    ##scaling function for substrate
    f=S/(Km+S)
    uptake=(v*ce/yA)*X1*f
    growth = (v*e-m*g)/(e+g)
    #Define derivatives
    dSdt = -uptake
    dedt = v*(f - e)
    dX1dt = X1*growth 
    dCO2dt = uptake*(1 - yA) + ce*(X1*e*(v-growth)) - growth*X1*MX1
    return dSdt, dedt, dX1dt, dCO2dt;

3. Defining outputs from DEB model

In [48]:
def calcDEB (model, pars, t, y0):
    #model parameters
    ##yA, Km, v, m, g, ce
    pars_model=pars[0:6]
    #conversion factors
    ##ce, nX1
    conversions=pars[5:7]
    
    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))
    #calculate B, WallsProto,
    B=((conversions[0]/4 + conversions[0]*y[:, 1])*y[:, 2])
    kec = (conversions[1]/4 + y[:, 1])/(0.25 + y[:, 1])
    Flush = B/kec
    Flush14C = Flush - Flush[0]
    
    #Create data with predictions
    yhat = np.concatenate((y[:, 0].reshape(len(d.Time),1), #Glucose
                           y[:, 3].reshape(len(d.Time),1),#CO2
                           kec.reshape(len(d.Time),1),
                           Flush14C.reshape(len(d.Time),1)), axis=1)
    
    return yhat

4. Defining objective function

In [49]:
def obj_funDEB (x):
    #define parameters
    ##yA, Km, v, m, g, ce, nX1, e
    pars = x
    #initial conditions
    S_i = d.Sinit[0]
    e_i = pars[7] #initial e is estimated as one model parameter
    X1_i = d.Cmicinit[0]/(pars[5]*(pars[6]/4 + e_i))*((0.25 + e_i)/(pars[6]/4 + e_i))
    
    y0 = np.array([S_i, e_i, X1_i, 0])
    #times
    t = d.Time
    #model simulations
    yhat_full = calcDEB(DEBmodel, pars, t, y0)
    #observations
    obs=np.concatenate((np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO214cumul]).reshape(len(d.Time),1),
                        np.array([d.kec]).reshape(len(d.Time),1),
                        np.array([d.Cmic14]).reshape(len(d.Time),1)), axis=1)
    #weights
    weights=np.concatenate((np.nanmean(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.CO214cumul).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.kec).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean((d.Cmic14)).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    out=np.nansum(((yhat_full-obs)/weights)**2)
    return out

5. Calculate the goodness of fit

In [5]:
def goodnessDEB (x):
    #define parameters
    ##yA, Km, v, m, g, ce, nX1, e
    pars = x
    #initial conditions
    S_i = d.Sinit[0]
    e_i = pars[7] #initial e is estimated as one model parameter
    X1_i = d.Cmicinit[0]/(pars[5]*(pars[6]/4 + e_i))*((0.25 + e_i)/(pars[6]/4 + e_i))
    
    y0 = np.array([S_i, e_i, X1_i, 0])
    #times
    t = d.Time
    #model simulations
    yhat_full = calcDEB(DEBmodel, pars, t, y0)
    #observations
    obs=np.concatenate((np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO214cumul]).reshape(len(d.Time),1),
                        np.array([d.kec]).reshape(len(d.Time),1),
                        np.array([d.Cmic14]).reshape(len(d.Time),1)), axis=1)
    
    R2=1-np.nansum((obs-yhat_full)**2)/np.nansum((obs-np.nanmean(obs))**2)
    ll=-np.nansum((obs-yhat_full)**2)/2/np.nanstd(obs)**2
    AIC = len(pars)*2 - 2*ll
    out = np.array([R2, ll, AIC])
    return out

Following function return the model solution for visualization in R

In [6]:
def predDEB (model, pars, t, y0):
    #model parameters
    ##yA, Km, v, m, g, ce
    pars_model=pars[0:6]
    #conversion factors
    ##ce, nX1
    conversions=pars[5:7]
    
    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))
    #calculate B, WallsProto,
    B=((conversions[0]/4 + conversions[0]*y[:, 1])*y[:, 2])
    kec = (conversions[1]/4 + y[:, 1])/(0.25 + y[:, 1])
    Flush = B/kec
    Flush14C = Flush - Flush[0]
    
    #Create data with predictions
    yhat = np.concatenate((y[:, 0].reshape(126,1), #Glucose
                           y[:, 3].reshape(126,1),#CO2
                           kec.reshape(len(d.Time),1),
                           Flush14C.reshape(len(d.Time),1)), axis=1)
    
    return yhat

6a. Reading data - Viden/Sucrose

In [7]:
d = pd.read_csv('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/SoilMBVariabilityData/VidenSucrose.csv', sep=',')
print(d)

   Unnamed: 0   Time         Rc         B        WP  Sinit
0           1    0.1   0.110000  2.130000  0.054500  51.29
1           2   24.0  15.831667  3.590000  0.359000  51.29
2           3   48.0  22.078333  3.723333  0.718833  51.29
3           4   72.0  24.503333  3.223333  0.624000  51.29
4           5   96.0  26.570000  2.593333  1.051833  51.29
5           6  120.0  27.845000  1.733333  0.496000  51.29


7a. Estimating parameters - Viden/Sucrose

In [8]:
VidenSucroseDA=dual_annealing(obj_funDEB, [(0.05, 1), (10, 1000),(0.001, 5), (1e-12, 0.1), (0.1, 3), (0.1, 10), (0, 1)])

<ipython-input-3-de0bdb1e2a70>:11: RuntimeWarning: invalid value encountered in true_divide
  WallsProto = (conversions[1]*conversions[0]/4)/(conversions[0]*y[:, 1])
/usr/local/lib/python3.8/dist-packages/scipy/integrate/odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


In [9]:
print(VidenSucroseDA)
print(goodnessDEB(VidenSucroseDA.x))

     fun: 1.3422467666650593
 message: ['Maximum number of iteration reached']
    nfev: 15657
    nhev: 0
     nit: 1000
    njev: 207
  status: 0
 success: True
       x: array([3.67434395e-01, 9.99999954e+02, 2.88215860e-01, 6.56791538e-02,
       1.56605948e-01, 5.35799179e+00, 7.02257631e-02])
[ 0.91402545 -0.77377095 15.54754189]


In [10]:
VidenSucroseDE=differential_evolution(obj_funDEB, [(0.05, 1), (10, 1000),(0.001, 5), (1e-12, 0.1), (0.1, 3), (0.1, 10), (0, 1)])

In [11]:
print(VidenSucroseDE)
print(goodnessDEB(VidenSucroseDE.x))

     fun: 1.3427115375467906
     jac: array([-3.55204755e-03, -7.94741419e-04, -3.48603369e-03,  2.28519426e-03,
       -3.89199784e-04,  1.75859328e-05, -1.58317803e-04])
 message: 'Optimization terminated successfully.'
    nfev: 13794
     nit: 129
 success: True
       x: array([3.64588512e-01, 9.97024271e+02, 2.85359016e-01, 6.56219526e-02,
       1.55736067e-01, 6.47628617e+00, 7.04446016e-02])
[ 0.91396394 -0.77432458 15.54864915]


In [12]:
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/VidenSucrosePars.csv', VidenSucroseDE.x.reshape(1,7), delimiter=",")

In [13]:
#initial conditions
S_i = d.Sinit[0]
e_i =VidenSucroseDE.x[6]/4/d.WP[0]
X1_i = d.B[0]/(VidenSucroseDE.x[5]*(0.25 + e_i))
    
y0 = np.array([S_i, e_i, X1_i, 0])
#times
t = np.arange(126)
#model simulations
VidenSucrosePred = predDEB(DEBmodel, VidenSucroseDE.x, t, y0)
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/VidenSucrosePred.csv', VidenSucrosePred, delimiter=",")

6b. Reading data - Viden/Sodium acetate

In [14]:
d = pd.read_csv('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/SoilMBVariabilityData/VidenSodiumAcetate.csv', sep=',')
print(d)

   Unnamed: 0   Time         Rc         B        WP  Sinit
0           1    0.1   0.166667  1.313333  0.054500  52.61
1           2   24.0   5.200000  1.826667  0.178167  52.61
2           3   48.0  10.148333  1.935000  0.147667  52.61
3           4   72.0  11.787500  2.020000  0.311000  52.61
4           5   96.0  12.358333  1.246667  0.173667  52.61
5           6  120.0  14.020000  2.445000  0.098167  52.61


7b. Estimating parameters - Viden/Sodium acetate

In [15]:
VidenSodiumAcetateDA=dual_annealing(obj_funDEB, [(0.05, 1), (10, 1000),(0.001, 5), (1e-12, 0.1), (0.1, 3), (0.1, 10), (0, 1)])

<ipython-input-3-de0bdb1e2a70>:11: RuntimeWarning: invalid value encountered in true_divide
  WallsProto = (conversions[1]*conversions[0]/4)/(conversions[0]*y[:, 1])


In [16]:
print(VidenSodiumAcetateDA)
print(goodnessDEB(VidenSodiumAcetateDA.x))

     fun: 1.9944034897067815
 message: ['Maximum number of iteration reached']
    nfev: 19857
    nhev: 0
     nit: 1000
    njev: 732
  status: 0
 success: True
       x: array([5.16372889e-01, 4.61199003e+02, 1.64977165e-01, 5.57882354e-02,
       2.46654094e-01, 2.72944310e+00, 6.04887670e-02])
[ 0.90707793 -0.83629863 15.67259725]


In [17]:
VidenSodiumAcetateDE=differential_evolution(obj_funDEB, [(0.05, 1), (10, 1000),(0.001, 5), (1e-12, 0.1), (0.1, 3), (0.1, 10), (0, 1)])

In [18]:
print(VidenSodiumAcetateDE)
print(goodnessDEB(VidenSodiumAcetateDE.x))

     fun: 1.9217311770334669
     jac: array([-4.66293671e-06, -8.19788031e-05,  3.89688282e-05, -3.93240995e-05,
       -1.95399252e-05, -2.44249067e-07, -3.79696274e-05])
 message: 'Optimization terminated successfully.'
    nfev: 13234
     nit: 121
 success: True
       x: array([3.13150356e-01, 9.98473616e+02, 1.75361728e-01, 3.86544735e-02,
       1.82794218e-01, 1.21078582e+00, 2.92311541e-02])
[ 0.9171784  -0.74539437 15.49078875]


In [19]:
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/VidenSodiumAcetatePars.csv', VidenSodiumAcetateDE.x.reshape(1,7), delimiter=",")

In [20]:
#initial conditions
S_i = d.Sinit[0]
e_i =VidenSodiumAcetateDE.x[6]/4/d.WP[0]
X1_i = d.B[0]/(VidenSodiumAcetateDE.x[5]*(0.25 + e_i))
    
y0 = np.array([S_i, e_i, X1_i, 0])
#times
t = np.arange(126)
#model simulations
VidenSodiumAcetatePred = predDEB(DEBmodel, VidenSodiumAcetateDE.x, t, y0)
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/VidenSodiumAcetatePred.csv', VidenSodiumAcetatePred, delimiter=",")

6c. Reading data - CB/Sucrose

In [21]:
d = pd.read_csv('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/SoilMBVariabilityData/CBSucrose.csv', sep=',')
print(d)

   Unnamed: 0   Time         Rc          B        WP  Sinit
0           1    0.1   0.106667  14.576667  2.270000  51.29
1           2   24.0   6.813333  45.540000  1.353333  51.29
2           3   48.0  20.923333  11.866667  3.146667  51.29
3           4   72.0  18.010000  12.840000  5.286667  51.29
4           5   96.0  18.853333  17.390000  2.793333  51.29
5           6  120.0  18.305000  13.113333  3.416667  51.29


7c. Estimating parameters - CB/Sucrose

In [22]:
CBSucroseDA=dual_annealing(obj_funDEB, [(0.05, 1), (10, 1000),(0.001, 5), (1e-12, 0.1), (0.1, 10), (0.1, 10), (0, 1)])

<ipython-input-3-de0bdb1e2a70>:11: RuntimeWarning: invalid value encountered in true_divide
  WallsProto = (conversions[1]*conversions[0]/4)/(conversions[0]*y[:, 1])


In [23]:
print(CBSucroseDA)
print(goodnessDEB(CBSucroseDA.x))

     fun: 4.103385916069149
 message: ['Maximum number of iteration reached']
    nfev: 38873
    nhev: 0
     nit: 1000
    njev: 3109
  status: 0
 success: True
       x: array([1.00000000e+00, 9.99985702e+02, 1.09708628e-01, 1.00000000e-12,
       4.25724505e+00, 7.79398540e+00, 4.36532627e-01])
[ 0.45024208 -4.9478213  23.89564259]


In [24]:
CBSucroseDE=differential_evolution(obj_funDEB, [(0.05, 1), (10, 1000),(0.001, 5), (1e-12, 0.1), (0.1, 10), (0.1, 10), (0, 1)])

In [25]:
print(CBSucroseDE)
print(goodnessDEB(CBSucroseDE.x))

     fun: 4.10558356365306
     jac: array([-1.87897696e-02, -4.85833210e-05,  4.08739709e-04,  1.77805854e+01,
        1.55964131e-03,  5.15143487e-06, -3.53850283e-04])
 message: 'Optimization terminated successfully.'
    nfev: 5402
     nit: 49
 success: True
       x: array([1.00000000e+00, 9.93292397e+02, 1.08204674e-01, 1.00000000e-12,
       5.97598975e+00, 7.63904990e+00, 4.43147189e-01])
[ 0.44817891 -4.96638977 23.93277954]


In [26]:
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/CBSucrose.csv', CBSucroseDE.x.reshape(1,7), delimiter=",")

8. Return the solution for visualization in R

In [27]:
#initial conditions
S_i = d.Sinit[0]
e_i =CBSucroseDE.x[6]/4/d.WP[0]
X1_i = d.B[0]/(CBSucroseDE.x[5]*(0.25 + e_i))
    
y0 = np.array([S_i, e_i, X1_i, 0])
#times
t = np.arange(126)
#model simulations
CBSucrosePred = predDEB(DEBmodel, CBSucroseDE.x, t, y0)
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/CBSucrosePred.csv', CBSucrosePred, delimiter=",")

6d. Reading data - CB/Amonnium acetate

In [28]:
d = pd.read_csv('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/SoilMBVariabilityData/CBAmonniumAcetate.csv', sep=',')
print(d)

   Unnamed: 0   Time         Rc          B        WP  Sinit
0           1    0.1   0.096667  14.240000  2.270000  51.28
1           2   24.0   2.720000  45.726667  1.871000  51.28
2           3   48.0   9.675000   7.910000  4.275000  51.28
3           4   72.0  10.980000  28.430000       NaN  51.28
4           5   96.0  10.263333  14.746667  4.013333  51.28
5           6  120.0   8.326667  20.703333  4.360000  51.28


7d. Estimating parameters - CB/Amonnium acetate

In [29]:
CBAmonniumAcetateDA=dual_annealing(obj_funDEB, [(0.05, 1), (10, 1000),(0.001, 5), (1e-12, 0.1), (0.1, 10), (0.1, 10), (0, 1)])

<ipython-input-3-de0bdb1e2a70>:11: RuntimeWarning: invalid value encountered in true_divide
  WallsProto = (conversions[1]*conversions[0]/4)/(conversions[0]*y[:, 1])


In [30]:
print(CBAmonniumAcetateDA)
print(goodnessDEB(CBAmonniumAcetateDA.x))

     fun: 3.554194339050463
 message: ['Maximum number of iteration reached']
    nfev: 46313
    nhev: 0
     nit: 1000
    njev: 4039
  status: 0
 success: True
       x: array([8.55066512e-01, 8.91539593e+02, 5.33586174e-02, 1.00000000e-12,
       5.95766300e-01, 1.60634127e+00, 6.86917438e-01])
[ 0.44498113 -4.7176604  23.4353208 ]


In [31]:
CBAmonniumAcetateDE=differential_evolution(obj_funDEB, [(0.05, 1), (10, 1000),(0.001, 5), (1e-12, 0.1), (0.1, 10), (0.1, 10), (0, 1)])

In [32]:
print(CBAmonniumAcetateDE)
print(goodnessDEB(CBAmonniumAcetateDE.x))

     fun: 3.5519661751913363
     jac: array([ 2.23367990e-03, -5.37347518e-05,  1.19158017e-03,  2.95541364e+01,
       -6.53472828e-03,  7.54951661e-07, -1.80539582e-02])
 message: 'Optimization terminated successfully.'
    nfev: 6281
     nit: 56
 success: True
       x: array([7.98422537e-01, 9.48037907e+02, 5.26144478e-02, 1.00000000e-12,
       5.52401501e-01, 5.91749001e+00, 6.45401462e-01])
[ 0.44615688 -4.70766653 23.41533306]


In [33]:
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/CBAmonniumAcetate.csv', CBAmonniumAcetateDA.x.reshape(1,7), delimiter=",")

8. Return the solution for visualization in R

In [34]:
#initial conditions
S_i = d.Sinit[0]
e_i =CBAmonniumAcetateDA.x[6]/4/d.WP[0]
X1_i = d.B[0]/(CBAmonniumAcetateDA.x[5]*(0.25 + e_i))
    
y0 = np.array([S_i, e_i, X1_i, 0])
#times
t = np.arange(126)
#model simulations
CBAmonniumAcetatePred = predDEB(DEBmodel, CBAmonniumAcetateDA.x, t, y0)
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/CBAmonniumAcetatePred.csv', CBAmonniumAcetatePred, delimiter=",")